# Ligat Ha'al Data Analysis Project

## Overview
This notebook collects and analyzes Israeli Premier League (Ligat Ha'al) data from multiple sources:
- **Wikipedia**: Match results (20 seasons)
- **Transfermarkt**: Attendance statistics by team and season

## Data Pipeline
1. **Environment Setup**: Configure paths and directories
2. **Match Data Collection**: Scrape Wikipedia for match-by-match results
3. **Attendance Data Collection**: Scrape Transfermarkt for stadium attendance
4. **Data Enrichment**: Calculate derived metrics (points, goal difference, etc.)
5. **Analysis & Visualization**: (To be added)

## Project Structure
```
ligat_haal_project/
├── data/
│   ├── raw/              # Original scraped data
│   ├── interim/          # Cleaned/enriched data
│   └── processed/        # Final datasets for analysis
├── notebooks/            # This notebook
└── reports/
    └── figures/          # Visualizations
```

## Requirements
- Python 3.8+
- pandas, requests, beautifulsoup4, lxml
- See `requirements.txt` for full list

---

## Installation (Optional)

Run this cell only if you need to install dependencies in your notebook environment. 

**Recommended**: Use a virtual environment and install from `requirements.txt`:
```bash
pip install -r ../requirements.txt
```

In [80]:
# Optional: install requirements (recommended to use requirements.txt)
# If you need to install dependencies in the notebook environment, uncomment one of the lines below.
# It's better to run these once in your environment or use a virtualenv and install from requirements.txt.
# pip install -r ../requirements.txt
# or (not recommended to run on every notebook execution):
# pip -q install pandas requests python-dateutil python-dotenv


# Environment Setup

This cell sets up all the necessary paths and environment variables for the notebook:
- Defines `ROOT`, `DATA_DIR`, `INTERIM_DIR`, `PROCESSED_DIR`, `FIG_DIR`
- Creates these directories if they don't exist
- Loads API keys from `.env` or `key.env` files (if available)

**Important**: Run this cell first before running any other cells in the notebook.

In [81]:
# === Environment Setup ===
import os
import sys
from pathlib import Path
import requests

# Try to import python-dotenv (optional, for loading .env files)
try:
    from dotenv import load_dotenv
    DOTENV_AVAILABLE = True
except ImportError:
    DOTENV_AVAILABLE = False
    print("ℹ python-dotenv not installed. Environment variables will be loaded from system only.")

# Feature flag: disable API-Sports section by default (we use Wikipedia + Transfermarkt)
USE_APISPORTS = False

# Define project root (parent of notebooks folder)
ROOT = Path.cwd().parent if Path.cwd().name == 'notebooks' else Path.cwd()
if not (ROOT / 'data').exists() and (ROOT.parent / 'data').exists():
    ROOT = ROOT.parent

# Define data directories
DATA_DIR = ROOT / 'data' / 'raw'
INTERIM_DIR = ROOT / 'data' / 'interim'
PROCESSED_DIR = ROOT / 'data' / 'processed'
FIG_DIR = ROOT / 'reports' / 'figures'

# Create directories if they don't exist
for directory in [DATA_DIR, INTERIM_DIR, PROCESSED_DIR, FIG_DIR]:
    directory.mkdir(parents=True, exist_ok=True)

print("✅ Directories configured:")
print(f"  • ROOT: {ROOT}")
print(f"  • DATA_DIR: {DATA_DIR}")
print(f"  • INTERIM_DIR: {INTERIM_DIR}")
print(f"  • PROCESSED_DIR: {PROCESSED_DIR}")
print(f"  • FIG_DIR: {FIG_DIR}")

# Try to load environment variables from .env or key.env files
APISPORTS_KEY = os.getenv('APISPORTS_KEY')

if DOTENV_AVAILABLE and not APISPORTS_KEY:
    # Try multiple locations for .env files
    env_candidates = [
        ROOT / 'notebooks' / 'key.env',
        ROOT / 'notebooks' / '.env',
        ROOT / 'key.env',
        ROOT / '.env',
    ]
    for env_path in env_candidates:
        if env_path.exists():
            load_dotenv(env_path)
            APISPORTS_KEY = os.getenv('APISPORTS_KEY')
            if APISPORTS_KEY:
                print(f"\n✅ Loaded API key from: {env_path}")
                break

# Setup API configuration (only if enabled)
BASE_URL = None
HEADERS = None
api_get = None

if USE_APISPORTS:
    if APISPORTS_KEY:
        BASE_URL = "https://v3.football.api-sports.io"
        HEADERS = {
            'x-apisports-key': APISPORTS_KEY
        }
        
        # Helper function for API calls
        def api_get(endpoint, params=None):
            """Make a GET request to API-Sports."""
            url = f"{BASE_URL}{endpoint}"
            response = requests.get(url, headers=HEADERS, params=params, timeout=30)
            response.raise_for_status()
            return response.json()
        
        print(f"✅ API configured with key: {APISPORTS_KEY[:4]}...{APISPORTS_KEY[-4:]}")
    else:
        print("\nℹ USE_APISPORTS=True but APISPORTS_KEY not found. API features will not work.")
else:
    # Provide a stub so accidental calls are obvious
    def api_get(*args, **kwargs):
        raise RuntimeError("API-Sports is disabled (set USE_APISPORTS=True in Environment Setup to enable)")
    print("\n⏸ API-Sports is disabled by default (USE_APISPORTS=False). Using Wikipedia + Transfermarkt pipeline.")

print("\n🎯 Environment setup complete! You can now run other cells.")

✅ Directories configured:
  • ROOT: c:\Users\nitib\dev-lab\ligat_haal_project\ligat_haal_project
  • DATA_DIR: c:\Users\nitib\dev-lab\ligat_haal_project\ligat_haal_project\data\raw
  • INTERIM_DIR: c:\Users\nitib\dev-lab\ligat_haal_project\ligat_haal_project\data\interim
  • PROCESSED_DIR: c:\Users\nitib\dev-lab\ligat_haal_project\ligat_haal_project\data\processed
  • FIG_DIR: c:\Users\nitib\dev-lab\ligat_haal_project\ligat_haal_project\reports\figures

⏸ API-Sports is disabled by default (USE_APISPORTS=False). Using Wikipedia + Transfermarkt pipeline.

🎯 Environment setup complete! You can now run other cells.


## Step 2: Enrich Match Data (2016/17 Example)

This cell demonstrates how to enrich raw match data with calculated metrics:
- **Goal difference**: home_goals - away_goals
- **Match result**: H (home win), A (away win), D (draw)
- **Points**: 3 for win, 1 for draw, 0 for loss
- **One-sided flag**: Matches with goal difference ≥ 3

**Input**: `data/raw/matches_2016_17_ligat_haal_wikipedia.csv`  
**Output**: `data/interim/matches_2016_17_ligat_haal_enriched.csv`

In [82]:
# Enrich Wikipedia match-by-match table (simplified, stable)
# Drops problematic rank/quartile fields and keeps only reliable derived columns
import pandas as pd
from pathlib import Path
import re

# Input/Output
matches_csv = Path(DATA_DIR) / "matches_2016_17_ligat_haal_wikipedia.csv"
out_path = INTERIM_DIR / "matches_2016_17_ligat_haal_enriched.csv"

# Safety checks
if not matches_csv.exists():
    raise FileNotFoundError(f"Matches CSV not found: {matches_csv} — run the Wikipedia scraping cell first")

# Load
df = pd.read_csv(matches_csv)

# Ensure numeric goal columns (in case of strings)
for col in ["home_goals", "away_goals"]:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors="coerce")

# Derived columns
df["goal_diff"] = df["home_goals"] - df["away_goals"]
df["result"] = df["goal_diff"].apply(lambda x: "H" if x > 0 else ("A" if x < 0 else "D"))
df["home_points"] = df["result"].map({"H": 3, "D": 1, "A": 0})
df["away_points"] = df["result"].map({"A": 3, "D": 1, "H": 0})

# Optional: simple flag for one-sided results
df["one_sided"] = (df["goal_diff"].abs() >= 3).astype(int)

# Reorder/keep columns defensively
cols = [
    "season", "home_team", "away_team",
    "home_goals", "away_goals", "goal_diff", "result",
    "home_points", "away_points", "one_sided"
]
ordered = [c for c in cols if c in df.columns]
df = df[ordered]

# Save
out_path.parent.mkdir(parents=True, exist_ok=True)
df.to_csv(out_path, index=False, encoding="utf-8-sig")
print(f"Saved enriched matches to: {out_path} | rows: {len(df)}")
display(df.head(10))

Saved enriched matches to: c:\Users\nitib\dev-lab\ligat_haal_project\ligat_haal_project\data\interim\matches_2016_17_ligat_haal_enriched.csv | rows: 182


,season,home_team,away_team,home_goals,away_goals,goal_diff,result,home_points,away_points,one_sided
0,2016/17,F.C. Ashdod,BEI,0,0,0,D,1,1,0
1,2016/17,F.C. Ashdod,BnS,1,1,0,D,1,1,0
2,2016/17,F.C. Ashdod,BnY,2,2,0,D,1,1,0
3,2016/17,F.C. Ashdod,HAS,1,0,1,H,3,0,0
4,2016/17,F.C. Ashdod,HBS,0,1,-1,A,0,3,0
5,2016/17,F.C. Ashdod,HHA,1,3,-2,A,0,3,0
6,2016/17,F.C. Ashdod,HKS,0,0,0,D,1,1,0
7,2016/17,F.C. Ashdod,HRA,0,1,-1,A,0,3,0
8,2016/17,F.C. Ashdod,HTA,1,0,1,H,3,0,0
9,2016/17,F.C. Ashdod,IKS,0,0,0,D,1,1,0


## Step 3: Advanced Enrichment (2022/23 Example)

This cell shows a more comprehensive enrichment process with:
- **Phase parsing**: Extract "regular", "championship", or "relegation" from round names
- **Round number**: Extract numeric round from strings like "Regular Season - 1"
- **Goal difference, results, points**: Same as Step 2
- **One-sided matches**: Flag matches with |goal_diff| ≥ 3
- **Column cleanup**: Remove irrelevant API-specific columns

**Input**: `data/raw/matches_2022_23_ligat_haal.csv` (if using API-Sports)  
**Output**: `data/interim/matches_2022_23_enriched.csv`

**Note**: This cell is for API-Sports data. For Wikipedia data, use the simpler enrichment in Step 2.

In [83]:
# === העשרת הטבלה + ניקוי עמודות מיותרות ===
import re
import pandas as pd

in_path  = DATA_DIR / "matches_2022_23_ligat_haal.csv"   # שנה לקובץ שלך
out_path = INTERIM_DIR / "matches_2022_23_enriched.csv"

if not in_path.exists():
    raise FileNotFoundError(f"Input matches file not found: {in_path}")

df = pd.read_csv(in_path)

# --- עמודות עזר ---
# 1) שנה מספרית לפתיחת העונה
#df["season_year"] = df["season"].str.slice(0,4).astype(int)

# 2) מספר מחזור ו-phase
def parse_round(r):
    # דוגמאות: "Regular Season - 1", "Championship Round - 5"
    if pd.isna(r):
        return (None, None)
    r = str(r)
    m = re.search(r"(Regular|Championship|Relegation).*?(\d+)", r, flags=re.I)
    phase = None
    if "regular" in r.lower():      phase = "regular"
    elif "championship" in r.lower(): phase = "championship"
    elif "relegation" in r.lower():   phase = "relegation"
    round_num = int(m.group(2)) if m else None
    return (phase, round_num)

tmp = df["round"].apply(parse_round).tolist()
df["phase"] = [t[0] for t in tmp]
df["round_num"] = [t[1] for t in tmp]

# 3) הפרש שערים, תוצאה, נקודות
df["goal_diff"] = df["home_goals"] - df["away_goals"]
df["result"] = df["goal_diff"].apply(lambda x: "H" if x>0 else ("A" if x<0 else "D"))
df["home_points"] = df["result"].map({"H":3, "D":1, "A":0})
df["away_points"] = df["result"].map({"H":0, "D":1, "A":3})

# 4) דגל משחק חד-צדדי (למשל |GD|>=3)
df["one_sided"] = (df["goal_diff"].abs() >= 3).astype(int)

# 5) עמודות לא רלוונטיות להסרה (כפי שביקשת)
drop_cols = ["league_id","league_name","fixture_id"]
df = df.drop(columns=[c for c in drop_cols if c in df.columns])

# 6) סדר עמודות נוח
cols = [
    "season","season_year","date","phase","round_num","stage",
    "home_team","away_team","home_goals","away_goals","goal_diff","result",
    "home_points","away_points","one_sided","venue","referee"
]
df = df[[c for c in cols if c in df.columns]]

df.to_csv(out_path, index=False, encoding="utf-8-sig")
print("נשמר:", out_path, "| שורות:", len(df))
df.head(10)


נשמר: c:\Users\nitib\dev-lab\ligat_haal_project\ligat_haal_project\data\interim\matches_2022_23_enriched.csv | שורות: 240


,season,date,phase,round_num,stage,home_team,away_team,home_goals,away_goals,goal_diff,result,home_points,away_points,one_sided,venue,referee
0,2022/23,2022-08-20,regular,1,Ligat Ha'al,Hapoel Haifa,Hapoel Tel Aviv,2,0,2,H,3,0,0,Sammy Ofer Stadium,O. Grinfeeld
1,2022/23,2022-08-20,regular,1,Ligat Ha'al,Hapoel Katamon,Hapoel Hadera,1,1,0,D,1,1,0,HaMoshava Stadium,A. Shiloach
2,2022/23,2022-08-20,regular,1,Ligat Ha'al,Maccabi Netanya,Beitar Jerusalem,4,1,3,H,3,0,1,Netanya Stadium,R. Reinshreiber
3,2022/23,2022-08-21,regular,1,Ligat Ha'al,Maccabi Tel Aviv,Maccabi Bnei Raina,5,0,5,H,3,0,1,Bloomfield Stadium,I. Frid
4,2022/23,2022-08-22,regular,1,Ligat Ha'al,Sektzia Nes Tziona,Ironi Kiryat Shmona,0,2,-2,A,0,3,0,HaMoshava Stadium,Y. Mizrahi
5,2022/23,2022-08-27,regular,2,Ligat Ha'al,Ironi Kiryat Shmona,Hapoel Katamon,1,1,0,D,1,1,0,Kiryat-Shmona Municipal Stadium,O. Na'al
6,2022/23,2022-08-27,regular,2,Ligat Ha'al,Hapoel Tel Aviv,Bnei Sakhnin,0,2,-2,A,0,3,0,Bloomfield Stadium,R. Reinshreiber
7,2022/23,2022-08-27,regular,2,Ligat Ha'al,Maccabi Haifa,Maccabi Netanya,4,1,3,H,3,0,1,Sammy Ofer Stadium,S. Levi
8,2022/23,2022-08-27,regular,2,Ligat Ha'al,Ashdod,Sektzia Nes Tziona,1,0,1,H,3,0,0,Yud-Alef Stadium,O. Asulin
9,2022/23,2022-08-28,regular,2,Ligat Ha'al,Maccabi Bnei Raina,Hapoel Haifa,1,1,0,D,1,1,0,Green Stadium,S. Ben Avraham


## Step 4: Scrape League Table from Wikipedia (2016/17)

This cell demonstrates how to fetch a league standings table from Wikipedia using pandas' `read_html()`.

**What it does**:
- Fetches the 2016/17 Israeli Premier League Wikipedia page
- Uses `read_html()` to automatically parse HTML tables
- Identifies the league table by looking for typical columns (Team, Points, etc.)
- Saves the standings to CSV

**Output**: `data/raw/ligat_haal_2016_17_wikipedia.csv`

**Note**: This gives you final standings, not match-by-match data. For match data, see the next cells.

In [84]:
# Scrape 2016/17 Ligat Ha'al league table from Wikipedia and save as CSV
# Use requests with a browser User-Agent to avoid HTTP 403 from the site
import re

url = "https://en.wikipedia.org/wiki/2016%E2%80%9317_Israeli_Premier_League"
resp = requests.get(url, headers={"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117 Safari/537.36"}, timeout=30)
resp.raise_for_status()

tables = pd.read_html(resp.text)

# Find the main league table (usually the first or second table)
def find_league_table(tables):
    for df in tables:
        # Look for columns typical of league tables
        cols = [c.lower() for c in df.columns.astype(str)]
        if any(re.search(r"team|club", c) for c in cols) and any(re.search(r"pts|points", c) for c in cols):
            return df
    return tables[0]  # fallback

league_df = find_league_table(tables)
print("Columns:", league_df.columns.tolist())
print("Rows:", len(league_df))

# Save to CSV in data/raw/
csv_path = Path(DATA_DIR) / "ligat_haal_2016_17_wikipedia.csv"
league_df.to_csv(csv_path, index=False, encoding="utf-8-sig")
print(f"Saved Wikipedia league table to: {csv_path}")
league_df.head()

Columns: ['Pos', 'Team', 'Pld', 'W', 'D', 'L', 'GF', 'GA', 'GD', 'Pts', 'Qualification or relegation']
Rows: 14
Saved Wikipedia league table to: c:\Users\nitib\dev-lab\ligat_haal_project\ligat_haal_project\data\raw\ligat_haal_2016_17_wikipedia.csv


C:\Users\nitib\AppData\Local\Temp\ipykernel_9016\2324587512.py:9: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(resp.text)


,Pos,Team,Pld,W,D,L,GF,GA,GD,Pts,Qualification or relegation
0,1,Hapoel Be'er Sheva,26,18,5,3,54,13,+41,59,Qualification for the Championship round
1,2,Maccabi Tel Aviv,26,17,5,4,45,19,+26,56,Qualification for the Championship round
2,3,Maccabi Petah Tikva,26,13,9,4,36,23,+13,48,Qualification for the Championship round
3,4,Beitar Jerusalem,26,10,10,6,34,27,+7,40,Qualification for the Championship round
4,5,Bnei Sakhnin,26,10,9,7,26,26,0,39,Qualification for the Championship round


## Step 5: Scrape Match-by-Match Results from Wikipedia (2016/17)

This cell extracts individual match results from Wikipedia's results matrix table.

**How it works**:
1. Fetches the Wikipedia page for 2016/17 season
2. Finds the results matrix table (grid showing Home vs Away results)
3. Parses each cell to extract scores (e.g., "2–1")
4. Creates one row per match with home/away teams and goals
5. Calculates derived metrics (goal_diff, result, points)

**Output**: `data/raw/matches_2016_17_ligat_haal_wikipedia.csv`

**Derived columns**:
- `goal_diff`: home_goals - away_goals
- `result`: H (home win), A (away win), D (draw)
- `home_points` / `away_points`: 3 for win, 1 for draw, 0 for loss

In [85]:
# Scrape 2016/17 Ligat Ha'al match-by-match results from Wikipedia and save as CSV
import pandas as pd
import requests
from bs4 import BeautifulSoup
from pathlib import Path
import re

# Fetch and parse the Wikipedia page
url = "https://en.wikipedia.org/wiki/2016%E2%80%9317_Israeli_Premier_League"
resp = requests.get(url, headers={"User-Agent": "Mozilla/5.0"}, timeout=30)
resp.raise_for_status()
soup = BeautifulSoup(resp.text, "lxml")

# Find the results matrix table by checking the first header cell
results_table = None
for table in soup.find_all("table", class_="wikitable"):
    first_row = table.find("tr")
    if first_row:
        first_cell = first_row.find("th")
        if first_cell and ("Home \\ Away" in first_cell.text or "Home / Away" in first_cell.text):
            results_table = table
            break

if not results_table:
    raise ValueError("Could not find results matrix table on Wikipedia page.")

# Parse teams from the first column and first row
rows = results_table.find_all("tr")
team_names = [td.get_text(strip=True) for td in rows[0].find_all("th")][1:]

# Build match list
matches = []
for i, row in enumerate(rows[1:]):
    cells = row.find_all(["th", "td"])
    home_team = cells[0].get_text(strip=True)
    for j, cell in enumerate(cells[1:]):
        away_team = team_names[j]
        score = cell.get_text(strip=True)
        # Only add if score looks like a result (e.g., '2–1')
        if re.match(r"^\d+\s*[–-]\s*\d+$", score):
            home_goals, away_goals = re.split(r"[–-]", score)
            matches.append({
                "season": "2016/17",
                "home_team": home_team,
                "away_team": away_team,
                "home_goals": int(home_goals.strip()),
                "away_goals": int(away_goals.strip()),
                "score": score
            })

# Convert to DataFrame and save
df = pd.DataFrame(matches)

# Add simple derived columns
df['goal_diff'] = df['home_goals'] - df['away_goals']
df['result'] = df['goal_diff'].apply(lambda x: "H" if x>0 else ("A" if x<0 else "D"))
df['home_points'] = df['result'].map({"H":3, "D":1, "A":0}).fillna(0).astype(int)
df['away_points'] = df['result'].map({"A":3, "D":1, "H":0}).fillna(0).astype(int)

# Select and order columns
keep_cols = ['season', 'home_team', 'away_team', 'home_goals', 'away_goals', 
             'goal_diff', 'result', 'home_points', 'away_points']
df = df[keep_cols]

# Save to CSV
csv_path = Path(DATA_DIR) / "matches_2016_17_ligat_haal_wikipedia.csv"
df.to_csv(csv_path, index=False, encoding="utf-8-sig")
print(f"Saved match-by-match results to: {csv_path}")
print(f"Total matches: {len(df)}")
df.head()

Saved match-by-match results to: c:\Users\nitib\dev-lab\ligat_haal_project\ligat_haal_project\data\raw\matches_2016_17_ligat_haal_wikipedia.csv
Total matches: 182


,season,home_team,away_team,home_goals,away_goals,goal_diff,result,home_points,away_points
0,2016/17,F.C. Ashdod,BEI,0,0,0,D,1,1
1,2016/17,F.C. Ashdod,BnS,1,1,0,D,1,1
2,2016/17,F.C. Ashdod,BnY,2,2,0,D,1,1
3,2016/17,F.C. Ashdod,HAS,1,0,1,H,3,0
4,2016/17,F.C. Ashdod,HBS,0,1,-1,A,0,3


## Step 6: Multi-Season Wikipedia Scraper (Last 20 Seasons)

This cell automates the match scraping process across multiple seasons.

**What it does**:
1. Calculates the last 20 seasons dynamically (based on current date)
2. For each season:
   - Fetches the Wikipedia page
   - Extracts the results matrix
   - Parses match-by-match data
   - Saves individual season CSV
3. Combines all seasons into one master file

**Outputs**:
- Per-season: `data/raw/matches_YYYY_YY_ligat_haal_wikipedia.csv`
- Combined: `data/raw/matches_all_seasons_ligat_haal_wikipedia.csv`

**Features**:
- Polite scraping with 1-second delays between requests
- Error handling for missing/changed pages
- Progress tracking with ✓/❌ indicators
- Season summary report

In [86]:
# Scrape multiple seasons of Ligat Ha'al from Wikipedia
import pandas as pd
import requests
from bs4 import BeautifulSoup
from pathlib import Path
import re
import time
from datetime import datetime

def scrape_season(season_year):
    """
    Scrape a single season's matches from Wikipedia.
    season_year: starting year (e.g., 2016 for 2016/17 season)
    """
    season_str = f"{season_year}/{str(season_year+1)[-2:]}"
    url = f"https://en.wikipedia.org/wiki/{season_year}%E2%80%93{str(season_year+1)[-2:]}_Israeli_Premier_League"
    
    print(f"Fetching {season_str}... ", end="", flush=True)
    try:
        resp = requests.get(url, headers={"User-Agent": "Mozilla/5.0"}, timeout=30)
        resp.raise_for_status()
        soup = BeautifulSoup(resp.text, "lxml")
        
        # Find results matrix
        results_table = None
        for table in soup.find_all("table", class_="wikitable"):
            first_row = table.find("tr")
            if first_row:
                first_cell = first_row.find("th")
                if first_cell and ("Home \\ Away" in first_cell.text or "Home / Away" in first_cell.text):
                    results_table = table
                    break
        
        if not results_table:
            print("❌ (no results matrix)")
            return None
            
        # Parse teams and build matches
        rows = results_table.find_all("tr")
        team_names = [td.get_text(strip=True) for td in rows[0].find_all("th")][1:]
        
        matches = []
        for i, row in enumerate(rows[1:]):
            cells = row.find_all(["th", "td"])
            home_team = cells[0].get_text(strip=True)
            for j, cell in enumerate(cells[1:]):
                away_team = team_names[j]
                score = cell.get_text(strip=True)
                if re.match(r"^\d+\s*[–-]\s*\d+$", score):
                    home_goals, away_goals = re.split(r"[–-]", score)
                    matches.append({
                        "season": season_str,
                        "season_year": season_year,
                        "home_team": home_team,
                        "away_team": away_team,
                        "home_goals": int(home_goals.strip()),
                        "away_goals": int(away_goals.strip())
                    })
        
        if not matches:
            print("❌ (no matches found)")
            return None
            
        # Convert to DataFrame and add derived columns
        df = pd.DataFrame(matches)
        df['goal_diff'] = df['home_goals'] - df['away_goals']
        df['result'] = df['goal_diff'].apply(lambda x: "H" if x>0 else ("A" if x<0 else "D"))
        df['home_points'] = df['result'].map({"H":3, "D":1, "A":0}).fillna(0).astype(int)
        df['away_points'] = df['result'].map({"A":3, "D":1, "H":0}).fillna(0).astype(int)
        
        # Select and order columns
        keep_cols = ['season', 'season_year', 'home_team', 'away_team', 'home_goals', 
                     'away_goals', 'goal_diff', 'result', 'home_points', 'away_points']
        df = df[keep_cols]
        
        print(f"✓ ({len(df)} matches)")
        return df
        
    except Exception as e:
        print(f"❌ ({str(e)[:50]}...)")
        return None

# List of seasons to scrape (last 20 seasons)
current_year = datetime.now().year
if datetime.now().month < 8:  # If before August, last season started in previous year
    current_year -= 1
seasons = list(range(current_year - 19, current_year + 1))

print(f"Scraping {len(seasons)} seasons from Wikipedia ({seasons[0]}/{str(seasons[0]+1)[-2:]} to {seasons[-1]}/{str(seasons[-1]+1)[-2:]})...")

# Scrape each season
all_matches = []
for season_year in seasons:
    df = scrape_season(season_year)
    if df is not None:
        # Save individual season
        season_path = DATA_DIR / f"matches_{season_year}_{str(season_year+1)[-2:]}_ligat_haal_wikipedia.csv"
        df.to_csv(season_path, index=False, encoding='utf-8-sig')
        all_matches.append(df)
    time.sleep(1)  # Be nice to Wikipedia

if all_matches:
    # Combine all seasons
    combined_df = pd.concat(all_matches, ignore_index=True)
    combined_path = DATA_DIR / "matches_all_seasons_ligat_haal_wikipedia.csv"
    combined_df.to_csv(combined_path, index=False, encoding='utf-8-sig')
    
    print("\nSummary:")
    print(f"- Successfully scraped {len(all_matches)} seasons")
    print(f"- Total matches: {len(combined_df)}")
    print(f"\nMatches per season:")
    season_counts = combined_df.groupby('season').size().sort_index()
    for season, count in season_counts.items():
        print(f"  • {season}: {count:3d} matches")
    print(f"\nAll matches saved to: {combined_path}")
    display(combined_df.head())

Scraping 20 seasons from Wikipedia (2006/07 to 2025/26)...
Fetching 2006/07... ✓ (132 matches)
✓ (132 matches)
Fetching 2007/08... Fetching 2007/08... ✓ (132 matches)
✓ (132 matches)
Fetching 2008/09... Fetching 2008/09... ✓ (132 matches)
✓ (132 matches)
Fetching 2009/10... Fetching 2009/10... ✓ (239 matches)
✓ (239 matches)
Fetching 2010/11... Fetching 2010/11... ✓ (234 matches)
✓ (234 matches)
Fetching 2011/12... Fetching 2011/12... ✓ (240 matches)
✓ (240 matches)
Fetching 2012/13... Fetching 2012/13... ✓ (182 matches)
✓ (182 matches)
Fetching 2013/14... Fetching 2013/14... ✓ (182 matches)
✓ (182 matches)
Fetching 2014/15... Fetching 2014/15... ✓ (181 matches)
✓ (181 matches)
Fetching 2015/16... Fetching 2015/16... ✓ (182 matches)
✓ (182 matches)
Fetching 2016/17... Fetching 2016/17... ✓ (182 matches)
✓ (182 matches)
Fetching 2017/18... Fetching 2017/18... ✓ (181 matches)
✓ (181 matches)
Fetching 2018/19... Fetching 2018/19... ✓ (182 matches)
✓ (182 matches)
Fetching 2019/20... Fetch

,season,season_year,home_team,away_team,home_goals,away_goals,goal_diff,result,home_points,away_points
0,2006/07,2006,Beitar Jerusalem,BnY,0,0,0,D,1,1
1,2006/07,2006,Beitar Jerusalem,ASH,2,0,2,H,3,0
2,2006/07,2006,Beitar Jerusalem,HAK,0,0,0,D,1,1
3,2006/07,2006,Beitar Jerusalem,HKS,2,0,2,H,3,0
4,2006/07,2006,Beitar Jerusalem,HPT,2,0,2,H,3,0


## Step 7: Scrape Attendance Data from Transfermarkt (Single Season)

This cell demonstrates scraping **actual attendance statistics** from Transfermarkt.

**Why Transfermarkt?**
- Wikipedia only shows stadium capacity (max seats), not actual attendance
- Transfermarkt provides real match attendance data aggregated by team per season

**Data collected per team**:
- `team`: Club name
- `average_attendance`: Average fans per home match
- `total_attendance`: Total fans across all home matches
- `stadium_capacity`: Maximum stadium capacity
- `utilization_pct`: Calculated as (average / capacity × 100)

**How utilization_pct is calculated**:
Since Transfermarkt doesn't provide a percentage column, we calculate it:
```
utilization_pct = (average_attendance / stadium_capacity) × 100
```

**Example output (2016/17)**:
- Hapoel Beer Sheva: 89.7% utilization (nearly full!)
- Ironi Kiryat Shmona: 10.8% utilization (mostly empty)

**Output**: `data/raw/attendance_YYYY_YY_ligat_haal_transfermarkt.csv`

**Source**: [Transfermarkt - Ligat Ha'al Attendance](https://www.transfermarkt.com/ligat-haal/besucherzahlen/wettbewerb/ISR1)

In [91]:
# Scrape attendance data from Transfermarkt for 2016/17 season (robust parser with header mapping)
import pandas as pd
import requests
from bs4 import BeautifulSoup
from pathlib import Path
import re
from io import StringIO
from typing import Optional, List

def _to_int(text: Optional[str]) -> Optional[int]:
    if text is None:
        return None
    s = str(text)
    s = s.replace('\xa0', ' ').strip()
    # remove everything that's not a digit
    digits = re.sub(r"[^0-9]", "", s)
    return int(digits) if digits else None

def _extract_club_from_cell(td: BeautifulSoup) -> Optional[str]:
    # Prefer anchors that link to a club page
    a = td.find('a', href=re.compile(r"/verein/|/club/", re.I))
    if a and a.get_text(strip=True):
        return a.get_text(strip=True)
    # Try image alt text (sometimes club crest has alt)
    img = td.find('img')
    if img and img.get('alt'):
        return img.get('alt').strip()
    # Sometimes the club appears in parentheses after stadium name
    txt = td.get_text(" ", strip=True)
    m = re.search(r"\(([^\)]+)\)$", txt)
    if m:
        return m.group(1).strip()
    # Fallback to text (stadium) if nothing else
    return txt

def scrape_transfermarkt_attendance(season_year: int) -> Optional[pd.DataFrame]:
    """
    Scrape attendance data from Transfermarkt for a specific season.
    season_year: starting year (e.g., 2016 for 2016/17 season)
    Returns: DataFrame with team, average_attendance, total_attendance, stadium_capacity, utilization_pct
    
    Note: utilization_pct is calculated as (average_attendance / stadium_capacity * 100)
    since Transfermarkt doesn't always provide this column directly.
    """
    season_str = f"{season_year}/{str(season_year+1)[-2:]}"
    url = f"https://www.transfermarkt.com/ligat-haal/besucherzahlen/wettbewerb/ISR1/saison_id/{season_year}"

    print(f"Fetching {season_str} from Transfermarkt... ", end="", flush=True)

    try:
        headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",
            "Accept-Language": "en-US,en;q=0.9,he;q=0.8,de;q=0.7",
            "Referer": "https://www.transfermarkt.com/",
            "Connection": "keep-alive",
        }
        sess = requests.Session()
        resp = sess.get(url, headers=headers, timeout=30)
        resp.raise_for_status()
        soup = BeautifulSoup(resp.text, "lxml")

        attendance_table = soup.find("table", class_="items")
        if not attendance_table:
            print("❌ (no attendance table found)")
            return None

        thead = attendance_table.find("thead")
        header_cells = []
        if thead:
            header_cells = [th.get_text(" ", strip=True).lower() for th in thead.find_all("th")]
        tbody = attendance_table.find("tbody") or attendance_table
        rows = tbody.find_all("tr")

        # Build a name->index mapping from header text
        header_index = {name: idx for idx, name in enumerate(header_cells)}
        def find_col_index(key_words: List[str]) -> Optional[int]:
            for name, idx in header_index.items():
                if any(kw in name for kw in key_words):
                    return idx
            return None

        idx_rank = find_col_index(["#"])  # not used
        idx_stadium = find_col_index(["stadium", "ground", "venue"]) or 1
        idx_capacity = find_col_index(["capacity", "kapaz"]) or 2
        idx_total = find_col_index(["spectators", "gesamt", "total"]) or 3
        idx_avg = find_col_index(["average", "durchschnitt", "ø"]) or 4
        idx_pct = find_col_index(["%", "percent", "auslast"])  # optional (usually not present)

        data: List[dict] = []
        for tr in rows:
            # Skip header or spacer rows
            if tr.find("th"):
                continue
            tds = tr.find_all("td")
            if not tds:
                continue

            # Compute offset if there are unlabeled extra columns (e.g., crest)
            offset = 0
            if header_cells and len(tds) != len(header_cells):
                # assume extra leading cells
                offset = max(0, len(tds) - len(header_cells))

            def td_at(index: int) -> Optional[BeautifulSoup]:
                i = index + offset
                return tds[i] if 0 <= i < len(tds) else None

            td_stadium = td_at(idx_stadium)
            td_capacity = td_at(idx_capacity)
            td_total = td_at(idx_total)
            td_avg = td_at(idx_avg)
            td_pct = td_at(idx_pct) if idx_pct is not None else None

            if not td_stadium:
                continue

            club = _extract_club_from_cell(td_stadium)
            capacity = _to_int(td_capacity.get_text()) if td_capacity else None
            total = _to_int(td_total.get_text()) if td_total else None
            avg = _to_int(td_avg.get_text()) if td_avg else None
            
            # Try to parse utilization if column exists (rarely the case)
            util = None
            if td_pct:
                util_text = td_pct.get_text(strip=True)
                util_digits = re.sub(r"[^0-9.,]", "", util_text).replace(',', '.')
                try:
                    util = float(util_digits) if util_digits else None
                except:
                    util = None

            # Filter out aggregate or empty rows
            if club and (avg is not None or total is not None or capacity is not None):
                # Ignore summary rows
                if str(club).lower() in {"total", "sum", "gesamt"}:
                    continue
                data.append({
                    "season": season_str,
                    "season_year": season_year,
                    "team": club,
                    "average_attendance": avg,
                    "total_attendance": total,
                    "stadium_capacity": capacity,
                    "utilization_pct": util,  # Will be recalculated below
                })

        if not data:
            print("❌ (parsed 0 usable rows)")
            return None

        df = pd.DataFrame(data)
        
        # Calculate utilization_pct from capacity and average if not already present
        # utilization_pct = (average_attendance / stadium_capacity) * 100
        if 'utilization_pct' in df.columns and df['utilization_pct'].isna().all():
            # Column exists but all None - calculate it
            df['utilization_pct'] = df.apply(
                lambda row: round((row['average_attendance'] / row['stadium_capacity'] * 100), 1) 
                if pd.notna(row['average_attendance']) and pd.notna(row['stadium_capacity']) and row['stadium_capacity'] > 0
                else None,
                axis=1
            )
        
        # Drop duplicates that may arise from multi-rows in table
        df = df.drop_duplicates(subset=["team"]).reset_index(drop=True)
        print(f"✓ ({len(df)} teams)")
        return df

    except Exception as e:
        print(f"❌ ({str(e)[:120]})")
        return None

# Test with 2016/17 season
season_year = 2016
df = scrape_transfermarkt_attendance(season_year)

if df is not None:
    # Save to CSV
    csv_path = DATA_DIR / f"attendance_{season_year}_{str(season_year+1)[-2:]}_ligat_haal_transfermarkt.csv"
    df.to_csv(csv_path, index=False, encoding='utf-8-sig')

    print(f"\nSaved to: {csv_path}")
    print(f"\nPreview:")
    display(df.head(20))

    if 'average_attendance' in df.columns and df['average_attendance'].notna().any():
        print(f"\nAttendance Summary for {season_year}/{str(season_year+1)[-2:]}:")
        print(f"  • Teams: {len(df)}")
        print(f"  • Average attendance (mean): {df['average_attendance'].mean():.0f}")
        print(f"  • Highest: {df.loc[df['average_attendance'].idxmax(), 'team']} ({df['average_attendance'].max():.0f})")
        print(f"  • Lowest: {df.loc[df['average_attendance'].idxmin(), 'team']} ({df['average_attendance'].min():.0f})")
        
        if 'utilization_pct' in df.columns and df['utilization_pct'].notna().any():
            print(f"\nStadium Utilization:")
            print(f"  • Average utilization: {df['utilization_pct'].mean():.1f}%")
            print(f"  • Highest: {df.loc[df['utilization_pct'].idxmax(), 'team']} ({df['utilization_pct'].max():.1f}%)")
            print(f"  • Lowest: {df.loc[df['utilization_pct'].idxmin(), 'team']} ({df['utilization_pct'].min():.1f}%)")
else:
    print("\n⚠ Could not retrieve attendance data from Transfermarkt.")
    print("Possible reasons:")
    print("  • Transfermarkt may not have data for this season")
    print("  • Website structure may have changed or is blocking bots")
    print("  • Network/access issue")
    print("\nTip: Try running again or switch to a fallback source (worldfootball.net)")

Fetching 2016/17 from Transfermarkt... ✓ (14 teams)

Saved to: c:\Users\nitib\dev-lab\ligat_haal_project\ligat_haal_project\data\raw\attendance_2016_17_ligat_haal_transfermarkt.csv

Preview:
✓ (14 teams)

Saved to: c:\Users\nitib\dev-lab\ligat_haal_project\ligat_haal_project\data\raw\attendance_2016_17_ligat_haal_transfermarkt.csv

Preview:


,season,season_year,team,average_attendance,total_attendance,stadium_capacity,utilization_pct
0,2016/17,2016,Maccabi Haifa,21891,284587,30780,71.1
1,2016/17,2016,Hapoel Beer Sheva,14467,188072,16126,89.7
2,2016/17,2016,Maccabi Tel Aviv,9529,123871,29150,32.7
3,2016/17,2016,Beitar Jerusalem,8192,106500,33500,24.5
4,2016/17,2016,Hapoel Tel Aviv,7900,102700,29150,27.1
5,2016/17,2016,Hapoel Haifa,4900,63700,30820,15.9
6,2016/17,2016,Maccabi Petah Tikva,3450,44850,11500,30.0
7,2016/17,2016,Bnei Yehuda Tel Aviv,3312,43050,6020,55.0
8,2016/17,2016,Ihud Bnei Sakhnin,3231,42000,8500,38.0
9,2016/17,2016,Hapoel Ashkelon,2785,36210,10000,27.9



Attendance Summary for 2016/17:
  • Teams: 14
  • Average attendance (mean): 6261
  • Highest: Maccabi Haifa (21891)
  • Lowest: Ironi Kiryat Shmona (1492)

Stadium Utilization:
  • Average utilization: 37.3%
  • Highest: Hapoel Beer Sheva (89.7%)
  • Lowest: Ironi Kiryat Shmona (10.8%)


## Step 8: Test Attendance Scraper (2023/24)

Quick validation test on a recent season to ensure the scraper works correctly.

**Output**: `data/raw/attendance_2023_24_ligat_haal_transfermarkt.csv`

In [ ]:
# Quick test: scrape 2023/24 season as well
season_year = 2023
_df_2023 = scrape_transfermarkt_attendance(season_year)
if _df_2023 is not None:
    _csv_2023 = DATA_DIR / f"attendance_{season_year}_{str(season_year+1)[-2:]}_ligat_haal_transfermarkt.csv"
    _df_2023.to_csv(_csv_2023, index=False, encoding='utf-8-sig')
    print(f"Saved to: {_csv_2023}")
    display(_df_2023.head(20))
else:
    print("Failed to scrape 2023/24 attendance from Transfermarkt.")
    # Quick test: scrape 2023/24 season as well


Fetching 2023/24 from Transfermarkt... ✓ (14 teams)
Saved to: c:\Users\nitib\dev-lab\ligat_haal_project\ligat_haal_project\data\raw\attendance_2023_24_ligat_haal_transfermarkt.csv
✓ (14 teams)
Saved to: c:\Users\nitib\dev-lab\ligat_haal_project\ligat_haal_project\data\raw\attendance_2023_24_ligat_haal_transfermarkt.csv


,season,season_year,team,average_attendance,total_attendance,stadium_capacity,utilization_pct
0,2023/24,2023,Maccabi Tel Aviv,17797,213565,29150,61.1
1,2023/24,2023,Maccabi Haifa,17195,171948,30780,55.9
2,2023/24,2023,Beitar Jerusalem,13166,144830,33500,39.3
3,2023/24,2023,Hapoel Beer Sheva,10169,122024,16126,63.1
4,2023/24,2023,Hapoel Tel Aviv,9186,101049,29150,31.5
5,2023/24,2023,Maccabi Netanya,5844,70127,13610,42.9
6,2023/24,2023,Hapoel Petah Tikva,5524,60759,11500,48.0
7,2023/24,2023,Hapoel Haifa,3869,42559,30820,12.6
8,2023/24,2023,Hapoel Jerusalem,3643,40070,33500,10.9
9,2023/24,2023,Maccabi Petah Tikva,3576,39337,11500,31.1


---

## Next Steps

Now that you have collected match and attendance data, you can:

### 1. Multi-Season Attendance Collection
Create a loop to scrape attendance for all 20 seasons (similar to the Wikipedia multi-season scraper).

### 2. Data Merging
Merge attendance data with match data by `(season, team)` to analyze:
- Home performance vs attendance levels
- Win rate correlation with fan support
- Derby match attendance spikes

### 3. Team Name Normalization
Standardize team names between Wikipedia and Transfermarkt sources for accurate joining.

### 4. Analysis & Visualization
- Time series of attendance trends
- Team performance over multiple seasons
- Home advantage analysis
- Goal-scoring patterns

### 5. Statistical Modeling
- Predict match outcomes based on historical data
- Attendance forecasting
- League position projections

---

**Current Data Available**:
- ✅ Match results: 20 seasons from Wikipedia (`matches_all_seasons_ligat_haal_wikipedia.csv`)
- ✅ Attendance: 2016/17 and 2023/24 from Transfermarkt
- ✅ Enriched data: Calculated metrics (points, goal_diff, results)

**Files Generated**:
```
data/raw/
├── matches_YYYY_YY_ligat_haal_wikipedia.csv (per season)
├── matches_all_seasons_ligat_haal_wikipedia.csv (combined)
├── attendance_2016_17_ligat_haal_transfermarkt.csv
└── attendance_2023_24_ligat_haal_transfermarkt.csv

data/interim/
├── matches_2016_17_ligat_haal_enriched.csv
└── matches_2022_23_enriched.csv
```